# Code for AISTATS submission (Reconstructing Test Labels from Noisy Loss Functions)
## Label inference with Itakura-Saito loss on IMDB

### Experiment setup

In [1]:
import os
import sys
import json
import pandas as pd
import numpy as np
from collections import Counter
import math
import time
import warnings
import random
warnings.filterwarnings('ignore')

np.random.seed(12345)

In [12]:
imdb_label = [0,1] * 12500
random.shuffle(imdb_label)

## utility functions in the experiments

In [13]:
# Method to compute binary log-loss
def itakura_loss(labeling = [], pred=[], tau=0):
    """
    labeling: ground truth labels
    pred: prediction vector
    Return itakura-saito score
    """
    if not pred:
        raise ValueError("No prediction vector passed")
    N = len(labeling)
    assert len(pred)==N
    loss = 0
    for i in range(N):
        theta0 = sys.float_info.min if 1 - pred[i] < sys.float_info.min else 1 - pred[i]
        loss += labeling[i] * (1/(1-theta0) + math.log(1-theta0) - 1)
        loss += (1 - labeling[i]) * (1/theta0 + math.log(theta0) - 1)
        
    return loss/N + tau * np.random.rand()

In [14]:
# function to generate the predictions
def generate_prediction(N, tau_max):
    """
    N: number of data points
    tau_max: maximum noise
    Return prediction vector
    """
    prediction = np.zeros(N)
    for i in range(N):
        n = i + 1
        exponent = 2 ** n * N * tau_max
        prediction[i] = 1 / (1 + 3 ** (-exponent))
    return list(prediction)

In [15]:
# generator for K-nary labelings
def gen_K_ary(nbits, K=2):
    """
    nbits: number of bits in the K-nary vector
    K: K in K-nary, e.g. K=2 is binary, K=3 is ternary
    Returns a generator for all K-nary labelings of length nbits
    """
    for i in range(K**nbits):
        bin_str = str(np.base_repr(i, K))
        l = len(bin_str)
        bin_arr = np.zeros(nbits)
        for j in range(l):
            bin_arr[nbits-j-1] = float(bin_str[l-j-1])
        yield [int(x) for x in bin_arr]

In [16]:
# the main label inference algorithm
def main_algorithm(noisy_loss, N, K, prediction):
    """
    noisy_loss: observed loss score with noise
    N: number of data points
    K: number of classes
    prediction: 2-d numpy array, where prediction[i,k] represents the prediction for data i in class k
    Returns the inferred labeling
    """
    label_infer = []
    diff_min = np.inf
    lab_generator = gen_K_ary(N, K)
    
    while True:
        try:
            label_cur = next(lab_generator) 
            loss_cur = itakura_loss(label_cur, prediction, 0)
            diff_cur = np.abs(loss_cur - noisy_loss)
            # print(loss_cur, diff_cur)
            if diff_cur < diff_min:
                label_infer = label_cur.copy()
                diff_min = diff_cur
        except:
            break
    return label_infer

In [17]:
# the main function for simulation
def main_sim(true_labeling, N, K, tau_max, nsims=1000):
    """
    true_labeling: ground truth labeling with length at least N
    N: number of data points
    K: number of classes
    tau_max: maximum noise
    nsims: number of simulations
    Returns the average accuracy and the average loss score
    """
    acc = 0
    mean_loss = 0
    for ns in range(nsims):
        labeling = list(np.random.choice(true_labeling, N))
        prediction = generate_prediction(N, tau_max)
        noisy_loss = itakura_loss(labeling, prediction, tau_max)
        mean_loss += noisy_loss
        labeling_infer = main_algorithm(noisy_loss, N, K, prediction)
        all_same = (sum([x == y for x, y in zip(labeling_infer, labeling)]) == N)
        acc += all_same
    return acc / nsims, mean_loss / nsims

## label inference experiments

In [ ]:
# logloss experiment 
nsims = 1000
result_itakura = []
K = 2

for tau_max in [1e-4, 1e-2, 1e-1, 1]:
    for N in range(1, 21):
        try:
            start = time.time()
            acc, mean_loss = main_sim(imdb_label, N, K, tau_max, nsims)
            seconds = time.time() - start
            result_itakura.append((K, tau_max, N, acc, mean_loss, seconds))
            print(K, tau_max, N, acc, mean_loss, seconds)
            if acc < 0.5: break
        except:
            break

2 0.0001 1 1.0 0.30690220216008435 3.2532191276550293
2 0.0001 2 1.0 0.3069093082301055 3.2938179969787598
2 0.0001 3 1.0 0.30691207371572626 3.418987989425659
2 0.0001 4 1.0 0.30689466413633654 3.6072161197662354
2 0.0001 5 1.0 0.30696797063404213 4.193435907363892
2 0.0001 6 1.0 0.30681317895012794 5.1659252643585205
2 0.0001 7 1.0 0.3077468203354434 7.5391480922698975
2 0.0001 8 1.0 0.3103663979381154 12.676691055297852
2 0.0001 9 1.0 0.3213988539717904 24.028834104537964
2 0.0001 10 1.0 0.37967937474230745 49.112571001052856
2 0.0001 11 1.0 0.7707649485408868 103.53831124305725
2 0.0001 12 1.0 9.345373114714103 221.5852279663086
2 0.0001 13 1.0 4872.416238976957 469.24240899086


In [11]:
df_itakura = pd.DataFrame(result_itakura)
df_itakura.columns = ['K', 'tau_max', 'N', 'acc', 'mean_loss', 'seconds_1k']
df_itakura['ms'] = df_itakura['seconds_1k'] # since 1000 repetitions
df_itakura.to_csv('itakura_binary.csv', index=False)
print(df_itakura.shape)
df_itakura.head(10)

(35, 7)


K  tau_max   N  acc  mean_loss  seconds_1k         ms
0  2   0.0001   1  1.0   0.306939    0.106770   0.106770
1  2   0.0001   2  1.0   0.307025    0.152001   0.152001
2  2   0.0001   3  1.0   0.307159    0.251989   0.251989
3  2   0.0001   4  1.0   0.307530    0.471528   0.471528
4  2   0.0001   5  1.0   0.308055    0.975714   0.975714
5  2   0.0001   6  1.0   0.309597    2.068109   2.068109
6  2   0.0001   7  1.0   0.312234    4.735019   4.735019
7  2   0.0001   8  1.0   0.320085    9.968048   9.968048
8  2   0.0001   9  1.0   0.340611   21.684825  21.684825
9  2   0.0001  10  1.0   0.427281   47.559090  47.559090

# multiquery experiments

In [18]:
# the main function for simulation
def main_sim_multi(true_labeling, K, tau_max, shard_size, max_N_feasible):
    """
    true_labeling: ground truth labeling
    K: number of classes
    tau_max: maximum noise
    loss_type: either logloss or sigmoidloss
    shard_size: number of data points in each shard
    max_N_feasible: largest N such that the label inference algorithm is feasible
    Returns the average accuracy and the number of shards
    """
    acc = 0
    mean_loss = 0
    n_total = len(true_labeling)  
    n_shard = int(np.ceil(n_total / shard_size))
    for j in range(n_shard):
        if j < n_shard - 1:
            cur_shard_size = shard_size
            labeling = true_labeling[j*shard_size : (j+1)*shard_size]
        else:
            cur_shard_size = n_total - j*shard_size
            labeling = true_labeling[j*shard_size : n_total]
        N = int(min(max_N_feasible, cur_shard_size))
        labeling_cur = labeling[:N]
        tau = 2 * np.exp(1) * tau_max / (np.exp(1) - 1)
        prediction = generate_prediction(N, tau)
        noisy_loss = itakura_loss(labeling_cur, prediction, tau_max)
        labeling_infer = main_algorithm(noisy_loss, N, K, prediction)
        acc += sum([x == y for x, y in zip(labeling_infer, labeling_cur)])
    return acc / n_total, n_shard

In [20]:
loss_type = 'itakura'
itakura_multi = []

# read in the max_N_feasible for each K and tau_max setting from previous experimental results
setting_itakura = pd.read_csv('itakura_binary.csv')
df = setting_itakura.dropna().groupby(['K','tau_max'])['N'].max()
df = df.to_frame()
df = df.reset_index()
df.head()

K  tau_max   N
0  2   0.0001  15
1  2   0.0100   9
2  2   0.1000   6
3  2   1.0000   5

In [21]:
prev_acc = 0
for _, rows in df.iterrows():
    K, tau_max, max_N_feasible = int(rows[0]), rows[1], int(rows[2])
    shard_size_vec = [1, 2, 5, 10, 50, 100, 500, 1000, 2500, 5000,12500, 25000]
    shard_size_vec = list(reversed(shard_size_vec))
    
    n_total = len(imdb_label)
    for i, shard_size in enumerate(shard_size_vec):
        start = time.time()
        acc, n_shards = main_sim_multi(imdb_label, K, tau_max, shard_size, max_N_feasible)
        if i > 0:
            acc = max(acc, prev_acc)
        seconds = time.time() - start
        itakura_multi.append((K, tau_max, acc, n_shards, seconds))
        print(K, tau_max, acc, n_shards, seconds)
        prev_acc = acc

2 0.0001 0.00036 1 2.0224030017852783
2 0.0001 0.00068 2 4.0651469230651855
2 0.0001 0.00152 5 10.17677116394043
2 0.0001 0.0034 10 20.51270294189453
2 0.0001 0.00868 25 49.85416221618652
2 0.0001 0.01784 50 99.5760850906372
2 0.0001 0.08652 250 512.9278709888458
2 0.0001 0.172 500 1160.4171619415283
2 0.0001 1.0 2500 121.57703614234924
2 0.0001 1.0 5000 5.215749263763428
2 0.0001 1.0 12500 1.1930549144744873
2 0.0001 1.0 25000 1.1131317615509033
2 0.01 0.00028 1 0.022813081741333008
2 0.01 0.00044 2 0.04352211952209473
2 0.01 0.00124 5 0.11215615272521973
2 0.01 0.00224 10 0.24277710914611816
2 0.01 0.005 25 0.568289041519165
2 0.01 0.01068 50 1.1374962329864502
2 0.01 0.0534 250 4.793082237243652
2 0.01 0.10756 500 9.40774393081665
2 0.01 0.53472 2500 47.17033100128174
2 0.01 1.0 5000 4.505929946899414
2 0.01 1.0 12500 1.082716941833496
2 0.01 1.0 25000 1.0578241348266602
2 0.1 0.00016 1 0.0018558502197265625
2 0.1 0.00032 2 0.0035822391510009766
2 0.1 0.0008 5 0.009776115417480469
2

In [22]:
itakura_multi = pd.DataFrame(itakura_multi)
itakura_multi.columns = ['K', 'tau_max', 'acc', 'n_shards', 'seconds']
itakura_multi.to_csv('itakura_multiquery_imdb.csv', index=False)
print(itakura_multi.shape)
itakura_multi.head(10)

(48, 5)


K  tau_max      acc  n_shards      seconds
0  2   0.0001  0.00036         1     2.022403
1  2   0.0001  0.00068         2     4.065147
2  2   0.0001  0.00152         5    10.176771
3  2   0.0001  0.00340        10    20.512703
4  2   0.0001  0.00868        25    49.854162
5  2   0.0001  0.01784        50    99.576085
6  2   0.0001  0.08652       250   512.927871
7  2   0.0001  0.17200       500  1160.417162
8  2   0.0001  1.00000      2500   121.577036
9  2   0.0001  1.00000      5000     5.215749